# **B5W1: Predicting Price Moves with News Sentiment**

## **Week 2**

### **Exploratory Data Analysis (EDA)**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import sys

In [18]:
sys.path.append(os.path.abspath(os.path.join('..')))

In [23]:
from scripts.RawAnalysis import load_data


df = load_data('raw_analyst_ratings/raw_analyst_ratings.csv')


ImportError: cannot import name 'load_data' from 'scripts.RawAnalysis' (c:\Users\nebe\B5W1_Predicting_Price_Moves_with_News_Sentiment\scripts\RawAnalysis.py)